# Surface Volume and CSV Processor

## Imports and Initializations

In [7]:
import os, re, json, yaml, math, sys, glob
from pathlib import Path
from typing import List, Tuple, Optional

In [8]:
CFG_PATH = os.path.join("..", "config.yml") if os.path.exists(os.path.join("..", "config.yml")) else "config.yml"
cfg = yaml.safe_load(open(CFG_PATH, "r", encoding="utf-8"))

PROJECT   = cfg["project_name"]
NUM_DIVS  = int(cfg["num_divs"])
DEM_DIR   = cfg["dem_folder"]
SV_OUT    = cfg["surface_volume_out"]

# Optional overrides (safe defaults provided if omitted)
SV_LEVELS = cfg.get("sv_heights_m")                    # e.g., [0.0, 0.5, 1.0, ...]  (meters, absolute water level)
SV_MIN    = cfg.get("sv_min_height_m", 0.0)            # used when sv_heights_m absent
SV_MAX    = cfg.get("sv_max_height_m", 10.0)
SV_STEP   = cfg.get("sv_step_height_m", 0.25)          # 25 cm default resolution
CSV_NAME  = cfg.get("sv_csv_name_fmt", f"{PROJECT}_div{{idx:02d}}.csv")
STRICT_INDEXING = bool(cfg.get("sv_strict_indexing", True))  # if True, enforce exactly NUM_DIVS files

In [11]:
def natural_key(s: str):
    """Sort like '..._0.tif', '..._1.tif', ..., '..._10.tif' numerically."""
    return [int(t) if t.isdigit() else t.lower() for t in re.findall(r'\d+|\D+', s)]

def extract_first_int(s: str) -> Optional[int]:
    m = re.search(r'(\d+)', Path(s).stem)
    return int(m.group(1)) if m else None

def build_levels() -> List[float]:
    if isinstance(SV_LEVELS, list) and len(SV_LEVELS) > 0:
        return [float(x) for x in SV_LEVELS]
    # fall back to range
    n = max(1, int(round((SV_MAX - SV_MIN) / SV_STEP)) + 1)
    return [SV_MIN + i*SV_STEP for i in range(n)]

LEVELS = build_levels()

def ensure_dir(p: str):
    Path(p).mkdir(parents=True, exist_ok=True)

def log(msg: str):
    print(msg, flush=True)

In [12]:
ARCPY_OK = False
try:
    import arcpy
    # 3D tools check
    if hasattr(arcpy, "CheckExtension") and arcpy.CheckExtension("3D") == "Available":
        ARCPY_OK = True
except Exception:
    ARCPY_OK = False

if not ARCPY_OK:
    import rasterio
    import numpy as np

# Ungrouped

In [13]:
from pathlib import Path
import os, glob

print("cwd:", os.getcwd())
print("DEM_DIR from config:", DEM_DIR)
p = Path(DEM_DIR)
print("DEM_DIR exists:", p.exists(), "abs:", p.resolve())

# Show a few files in the folder (any case)
cands = []
for pat in ["*.tif","*.TIF","*.tiff","*.TIFF","*.img","*.IMG","*.vrt","*.VRT","**/*.tif","**/*.TIF","**/*.vrt","**/*.VRT"]:
    cands += glob.glob(str(p / pat), recursive=True)
print("Found", len(cands), "raster-like files")
print(sorted(cands)[:10])

cwd: /mnt/e/CERA/GISSR/GIS_FloodSimulation/Generalized
DEM_DIR from config: /mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36
DEM_DIR exists: True abs: /mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36
Found 74 raster-like files
['/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36/LM_dem_merged.vrt', '/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36/LM_dem_merged.vrt', '/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36/new_div_label_0.tif', '/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36/new_div_label_0.tif', '/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36/new_div_label_1.tif', '/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36/new_div_label_1.tif', '/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36/new_div_label_10.tif', '/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36/new_div_label_10.tif', '/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36/new_div_label_11.tif', '/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36/new_div_label_11.tif']


In [14]:
def list_dem_files(folder: str) -> List[str]:
    # Common raster extensions
    # pats = ["*.tif", "*.tiff", "*.img", "*.vrt"]
    pats = ["*.tif"]
    paths = []
    for p in pats:
        paths.extend(glob.glob(str(Path(folder) / p)))
    # Prefer numeric order by index in filename if present
    with_idx = [(p, extract_first_int(p)) for p in paths]
    if any(idx is not None for _, idx in with_idx):
        with_idx = [(p, (idx if idx is not None else 10**9)) for p, idx in with_idx]
        with_idx.sort(key=lambda t: t[1])
        return [p for p, _ in with_idx]
    # else natural alphabetical
    return sorted(paths, key=natural_key)

DEM_PATHS = list_dem_files(DEM_DIR)

if STRICT_INDEXING and len(DEM_PATHS) != NUM_DIVS:
    raise RuntimeError(f"[Config mismatch] Found {len(DEM_PATHS)} DEM(s) in '{DEM_DIR}', but num_divs={NUM_DIVS}.")

if len(DEM_PATHS) == 0:
    raise RuntimeError(f"No DEM rasters found in '{DEM_DIR}'. Check folder and file extensions.")

In [15]:
# 0..3 at 0.25m
levels_small = np.round(np.arange(0.0, 3.0 + 0.25, 0.25), 2)      # 13 values
# 0..10 at 0.5m (we'll keep 3.5..7 from this)
levels_large = np.round(np.arange(0.0, 10.0 + 0.5, 0.5), 2)       # 21 values
# Final combined set expected by flood code: 0..3 (0.25) + 3.5..7 (0.5)
levels_comb  = np.concatenate([levels_small,
                               np.round(np.arange(3.5, 7.0 + 0.5, 0.5), 2)])  # 21 rows total

In [16]:
def surface_volume_arcgis(dem_path: str, heights_m: List[float]) -> List[Tuple[float, float, float, int]]:
    """
    Returns list of (height_m, area_m2, volume_m3, cell_count_placeholder)
    ArcGIS SurfaceVolume_3d outputs area/volume BELOW plane (Z = height), which matches flood volume to that level.
    """
    out = []
    arcpy.CheckOutExtension("3D")
    for h in heights_m:
        # SurfaceVolume_3d(in_raster, out_table, method, plane_height, ref_plane, z_factor)
        # method: "BELOW" for volume below a plane
        tbl = arcpy.CreateUniqueName("svtbl", "in_memory")
        try:
            arcpy.ddd.SurfaceVolume(dem_path, tbl, "BELOW", h, "HORIZONTAL_PLANE", 1.0)
            # Table has fields: AREA_2D, AREA_3D, VOLUME
            area = 0.0
            vol  = 0.0
            with arcpy.da.SearchCursor(tbl, ["AREA_2D", "VOLUME"]) as cur:
                for a2d, v in cur:
                    area += float(a2d or 0.0)
                    vol  += float(v or 0.0)
            out.append((float(h), float(area), float(vol), -1))
        finally:
            try:
                arcpy.management.Delete(tbl)
            except Exception:
                pass
    return out

def pixel_area_from_transform(transform) -> float:
    # Assumes near-orthogonal geotransform
    # area = |a * e - b * d| where Affine(a,b,c,d,e,f)
    return abs(transform.a * transform.e - transform.b * transform.d)

def surface_volume_numpy(dem_path: str, heights_m: List[float]) -> List[Tuple[float, float, float, int]]:
    """
    Compute flood 'surface volume' as volume of water required to fill terrain up to level h (m),
    and 2D surface area (planimetric) of wetted region. DEM and heights are in meters.
    """
    with rasterio.open(dem_path) as ds:
        if ds.crs is None:
            raise RuntimeError(f"{dem_path} has no CRS. Reproject to a projected CRS in meters.")
        if "degree" in (ds.crs.linear_units or "").lower():
            raise RuntimeError(f"{dem_path} is in degrees. Reproject to a projected CRS (meters) first.")
        nodata = ds.nodata
        T = ds.transform
        A = pixel_area_from_transform(T)
        elev = ds.read(1, masked=False).astype("float64")

    if nodata is not None:
        mask = elev == nodata
        elev[mask] = np.nan

    out = []
    finite = np.isfinite(elev)
    for h in heights_m:
        wet = finite & (elev < h)
        cells = int(np.count_nonzero(wet))
        if cells == 0:
            out.append((float(h), 0.0, 0.0, 0))
            continue
        # area (m^2) = wet cells * pixel area
        area_m2 = cells * A
        # volume (m^3) = sum((h - z) over wet) * pixel_area
        vol_m3 = float(np.nansum((h - elev[wet])) * A)
        out.append((float(h), float(area_m2), float(vol_m3), cells))
    return out

In [17]:
def write_csv(rows: List[Tuple[float,float,float,int]], out_csv: str):
    import csv
    ensure_dir(Path(out_csv).parent.as_posix())
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["height_m", "area_m2", "volume_m3", "cell_count"])
        for h,a,v,c in rows:
            w.writerow([f"{h:.6f}", f"{a:.6f}", f"{v:.6f}", c])

In [29]:
# Main Loop
def main():
    log("=== Surface Volume → CSV ===")
    log(f"Config: {PROJECT=}  {NUM_DIVS=}  DEM_DIR='{DEM_DIR}'  OUT='{SV_OUT}'")
    log(f"Levels: {len(LEVELS)} from {LEVELS[0]:.3f} to {LEVELS[-1]:.3f} m" + (f"  (custom)" if cfg.get("sv_heights_m") else ""))
    log(f"Engine: {'ArcGIS 3D Analyst' if ARCPY_OK else 'NumPy+rasterio'}")

    manifest = {
        "project": PROJECT,
        "num_divs": NUM_DIVS,
        "dem_dir": DEM_DIR,
        "sv_out": SV_OUT,
        "levels_m": LEVELS,
        "engine": "arcpy.SurfaceVolume" if ARCPY_OK else "numpy_rasterio",
        "items": []
    }

    if STRICT_INDEXING and len(DEM_PATHS) != NUM_DIVS:
        log(f"[WARN] Found {len(DEM_PATHS)} DEM(s); num_divs={NUM_DIVS}. Proceeding anyway due to STRICT_INDEXING={STRICT_INDEXING}.")

    for idx, dem_path in enumerate(DEM_PATHS):
        # If STRICT_INDEXING, force idx to be the loop index 0..NUM_DIVS-1
        target_idx = idx if STRICT_INDEXING else (extract_first_int(dem_path) or idx)
        out_csv = Path(SV_OUT) / CSV_NAME.format(idx=target_idx)

        log(f"[{idx+1}/{len(DEM_PATHS)}] {Path(dem_path).name}  ->  {out_csv.name}")

        try:
            if ARCPY_OK:
                rows = surface_volume_arcgis(dem_path, LEVELS)
            else:
                rows = surface_volume_numpy(dem_path, LEVELS)
            write_csv(rows, out_csv.as_posix())

            manifest["items"].append({
                "index": int(target_idx),
                "dem": dem_path,
                "csv": out_csv.as_posix(),
                "min_height_m": float(LEVELS[0]),
                "max_height_m": float(LEVELS[-1]),
                "n_levels": len(LEVELS)
            })
        except Exception as e:
            log(f"  !! ERROR on {dem_path}: {e}")
            manifest["items"].append({
                "index": int(target_idx),
                "dem": dem_path,
                "csv": None,
                "error": str(e)
            })

    # Sort manifest by index for convenience
    manifest["items"].sort(key=lambda d: d.get("index", 10**9))
    ensure_dir(SV_OUT)
    man_path = Path(SV_OUT) / f"{PROJECT}_sv_manifest.json"
    with open(man_path, "w", encoding="utf-8") as f:
        json.dump(manifest, f, indent=2)
    log(f"Done. Wrote {len([it for it in manifest['items'] if it.get('csv')])} CSV(s).")
    log(f"Manifest: {man_path}")

if __name__ == "__main__":
    main()

=== Surface Volume → CSV ===
Config: PROJECT='LM_div36'  NUM_DIVS=36  DEM_DIR='/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36'  OUT='/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/SurfaceVolume_LM_div36'
Levels: 41 from 0.000 to 10.000 m
Engine: NumPy+rasterio


[1/36] new_div_label_0.tif  ->  LM_div36_div00.csv
[2/36] new_div_label_1.tif  ->  LM_div36_div01.csv
[3/36] new_div_label_2.tif  ->  LM_div36_div02.csv
[4/36] new_div_label_3.tif  ->  LM_div36_div03.csv
[5/36] new_div_label_4.tif  ->  LM_div36_div04.csv
[6/36] new_div_label_5.tif  ->  LM_div36_div05.csv
[7/36] new_div_label_6.tif  ->  LM_div36_div06.csv
[8/36] new_div_label_7.tif  ->  LM_div36_div07.csv
[9/36] new_div_label_8.tif  ->  LM_div36_div08.csv
[10/36] new_div_label_9.tif  ->  LM_div36_div09.csv
[11/36] new_div_label_10.tif  ->  LM_div36_div10.csv
[12/36] new_div_label_11.tif  ->  LM_div36_div11.csv
[13/36] new_div_label_12.tif  ->  LM_div36_div12.csv
[14/36] new_div_label_13.tif  ->  LM_div36_div13.csv
[15/36] new_div_label_14.tif  ->  LM_div36_div14.csv
[16/36] new_div_label_15.tif  ->  LM_div36_div15.csv
[17/36] new_div_label_16.tif  ->  LM_div36_div16.csv
[18/36] new_div_label_17.tif  ->  LM_div36_div17.csv
[19/36] new_div_label_18.tif  ->  LM_div36_div18.csv
[20/36] new_

# Grouped Divs

In [3]:
import json, yaml, sys
cfg = yaml.safe_load(open("../config.yml"))
N = cfg["num_divs"]

def check_ids(L, name):
    bad = [i for i in L if not isinstance(i,int) or i<0 or i>=N]
    if bad: raise ValueError(f"{name}: out-of-range IDs {bad}")

if cfg["adjacency"]["type"] == "groups":
    G = json.load(open(cfg["adjacency"]["groups_file"]))
    groups = G["groups"] if isinstance(G, dict) else [{"order": g} for g in G]
    seen = set()
    for gi,g in enumerate(groups):
        order = g["order"]
        check_ids(order, f"group[{gi}].order")
        if len(set(order))!=len(order):
            raise ValueError(f"group[{gi}] has duplicates")
        overlap = set(order)&seen
        if overlap:
            raise ValueError(f"IDs appear in multiple groups: {sorted(overlap)}")
        seen |= set(order)
    if isinstance(G, dict) and "landlocked" in G:
        check_ids(G["landlocked"], "landlocked")
    print("Groups.json OK")
else:
    A = json.load(open(cfg["adjacency"]["graph_file"]))
    edges = A["edges"]
    if edges and isinstance(edges[0], list):   # simple form
        for u,v in edges:
            check_ids([u,v],"edge")
    else:
        for e in edges:
            check_ids([e["u"],e["v"]],"edge")
            if e.get("blocked") not in (None,True,False):
                raise ValueError("blocked must be true/false if present")
    if "num_nodes" in A and A["num_nodes"] != N:
        raise ValueError(f"num_nodes mismatch: {A['num_nodes']} vs {N}")
    if "landlocked" in A:
        check_ids(A["landlocked"], "landlocked")
    print("Adjacency.json OK")

Groups.json OK


In [4]:
# ---- Group-aware switches (optional) ----
SV_USE_GROUPS       = bool(cfg.get("sv_use_groups", False))
SV_ONLY_GROUPS      = bool(cfg.get("sv_only_groups", False))
SV_ORDER_BY_GROUPS  = bool(cfg.get("sv_order_by_groups", SV_ONLY_GROUPS))  # default: true if only_groups is true

def _normalize_groups(obj):
    """
    Accept both:
      - {"groups":[{"name":"A","order":[0,1,2]}, {"name":"B","order":[10,11]}], "landlocked":[...]}
      - [[0,1,2],[10,11]]
    Return: list of lists (orders), landlocked (list)
    """
    if isinstance(obj, dict):
        groups = obj.get("groups", [])
        if groups and isinstance(groups[0], dict):
            orders = [g["order"] for g in groups]
        else:
            orders = groups  # if already [[...],[...]]
        landlocked = obj.get("landlocked", [])
    else:
        orders = obj
        landlocked = []
    # flatten sanity
    flat = [i for g in orders for i in g]
    if len(set(flat)) != len(flat):
        dupes = sorted([i for i in flat if flat.count(i) > 1])
        raise ValueError(f"Groups.json contains duplicate IDs across groups: {dupes}")
    return orders, landlocked

def load_groups_from_cfg(cfg, N):
    """
    Read groups if cfg['adjacency']['type'] == 'groups'.
    Returns: list_of_orders, landlocked, coastal_ids_flat
    """
    adj = cfg.get("adjacency", {})
    if adj.get("type") != "groups":
        return None, [], []

    import json, os
    path = adj.get("groups_file")
    if not path or not os.path.exists(path):
        raise FileNotFoundError(f"adjacency.type='groups' but groups_file not found: {path}")
    obj = json.load(open(path, "r", encoding="utf-8"))
    orders, landlocked = _normalize_groups(obj)

    # validate ranges
    bad = [i for i in [x for g in orders for x in g] if (not isinstance(i, int)) or i < 0 or i >= N]
    if bad:
        raise ValueError(f"Groups.json has out-of-range IDs (0..{N-1}): {sorted(set(bad))}")

    return orders, landlocked, [i for g in orders for i in g]


In [9]:
def extract_first_int(s: str):
    import re
    m = re.search(r'(\d+)', Path(s).stem)
    return int(m.group(1)) if m else None

def list_dem_files(folder: str):
    import glob
    root = Path(folder)
    pats = ["*.tif","*.tiff","*.img","*.vrt","**/*.tif","**/*.tiff","**/*.img","**/*.vrt",
            "*.TIF","*.TIFF","*.IMG","*.VRT","**/*.TIF","**/*.TIFF","**/*.IMG","**/*.VRT"]
    paths = []
    for pat in pats:
        paths.extend(glob.glob(str(root / pat), recursive=True))
    # drop sidecars
    paths = [p for p in paths if not p.lower().endswith(".aux.xml")]
    return sorted(set(paths))

DEM_PATHS = list_dem_files(DEM_DIR)

if len(DEM_PATHS) == 0:
    raise RuntimeError(f"No DEM rasters found in '{DEM_DIR}'.")

# Map index -> path:
# priority 1: use numeric index embedded in filename (e.g., *_12.tif)
# priority 2: if STRICT_INDEXING, assign in sorted order (0..NUM_DIVS-1)
IDX_TO_DEM = {}
by_num = {}
for p in DEM_PATHS:
    k = extract_first_int(p)
    if k is not None:
        by_num[k] = p

if len(by_num) >= min(NUM_DIVS, len(DEM_PATHS)):
    # filenames carry indices; use them
    IDX_TO_DEM = by_num
else:
    # fallback: sequential mapping
    if STRICT_INDEXING and len(DEM_PATHS) != NUM_DIVS:
        raise RuntimeError(f"[Config mismatch] Found {len(DEM_PATHS)} DEM(s) in '{DEM_DIR}', but num_divs={NUM_DIVS}.")
    for i, p in enumerate(sorted(DEM_PATHS)):
        IDX_TO_DEM[i] = p

In [20]:
GROUP_ORDERS, GROUP_LANDLOCKED, COASTAL_IDS = (None, [], [])
if SV_USE_GROUPS:
    GROUP_ORDERS, GROUP_LANDLOCKED, COASTAL_IDS = load_groups_from_cfg(cfg, NUM_DIVS)

# Decide the processing list of indices
if SV_USE_GROUPS and SV_ONLY_GROUPS:
    PROCESS_IDX = list(COASTAL_IDS)  # only those in groups
else:
    # all indices that we have DEMs for
    PROCESS_IDX = sorted(IDX_TO_DEM.keys())

# Apply ordering preference
if SV_USE_GROUPS and SV_ORDER_BY_GROUPS and GROUP_ORDERS:
    # concatenate groups in the file order
    ordered = []
    for order in GROUP_ORDERS:
        ordered.extend(order)
    # retain only those we intend to process
    PROCESS_IDX = [i for i in ordered if i in set(PROCESS_IDX)]
# else leave as is (sorted by index)

In [21]:
def main():
    log("=== Surface Volume → CSV ===")
    log(f"Config: PROJECT={PROJECT}  NUM_DIVS={NUM_DIVS}  DEM_DIR='{DEM_DIR}'  OUT='{SV_OUT}'")
    log(f"Levels: {len(LEVELS)} from {LEVELS[0]:.3f} to {LEVELS[-1]:.3f} m")
    log(f"Engine: {'ArcGIS 3D Analyst' if ARCPY_OK else 'NumPy+rasterio'}")
    if SV_USE_GROUPS:
        log(f"Groups: use_groups={SV_USE_GROUPS} only_groups={SV_ONLY_GROUPS} order_by_groups={SV_ORDER_BY_GROUPS}")
        if GROUP_ORDERS:
            log(f"  #groups={len(GROUP_ORDERS)}  coastal_ids={len(COASTAL_IDS)}")

    manifest = {
        "project": PROJECT,
        "num_divs": NUM_DIVS,
        "dem_dir": DEM_DIR,
        "sv_out": SV_OUT,
        "levels_m": LEVELS,
        "engine": "arcpy.SurfaceVolume" if ARCPY_OK else "numpy_rasterio",
        "use_groups": SV_USE_GROUPS,
        "only_groups": SV_ONLY_GROUPS,
        "order_by_groups": SV_ORDER_BY_GROUPS,
        "groups": GROUP_ORDERS if GROUP_ORDERS is not None else [],
        "landlocked": GROUP_LANDLOCKED,
        "items": []
    }

    total = len(PROCESS_IDX)
    for k, idx in enumerate(PROCESS_IDX, 1):
        if idx not in IDX_TO_DEM:
            log(f"[{k}/{total}] idx={idx}: no DEM found; skipping")
            manifest["items"].append({"index": int(idx), "dem": None, "csv": None, "error": "missing DEM"})
            continue

        dem_path = IDX_TO_DEM[idx]
        out_csv = Path(SV_OUT) / CSV_NAME.format(idx=idx)

        log(f"[{k}/{total}] div {idx:02d}: {Path(dem_path).name} -> {out_csv.name}")
        try:
            rows = surface_volume_arcgis(dem_path, LEVELS) if ARCPY_OK else surface_volume_numpy(dem_path, LEVELS)
            write_csv(rows, out_csv.as_posix())
            manifest["items"].append({
                "index": int(idx),
                "dem": dem_path,
                "csv": out_csv.as_posix(),
                "min_height_m": float(LEVELS[0]),
                "max_height_m": float(LEVELS[-1]),
                "n_levels": len(LEVELS)
            })
        except Exception as e:
            log(f"  !! ERROR on idx {idx}: {e}")
            manifest["items"].append({"index": int(idx), "dem": dem_path, "csv": None, "error": str(e)})

    ensure_dir(SV_OUT)
    man_path = Path(SV_OUT) / f"{PROJECT}_sv_manifest.json"
    with open(man_path, "w", encoding="utf-8") as f:
        json.dump(manifest, f, indent=2)
    ok_cnt = sum(1 for it in manifest["items"] if it.get("csv"))
    log(f"Done. Wrote {ok_cnt}/{total} CSV(s).")
    log(f"Manifest: {man_path}")


In [22]:
if __name__ == "__main__":
    main()

=== Surface Volume → CSV ===
Config: PROJECT=LM_div36  NUM_DIVS=36  DEM_DIR='/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/LM_div36'  OUT='/mnt/e/CERA/GISSR/GIS_FloodSimulation/Data/SurfaceVolume_LM_div36'
Levels: 41 from 0.000 to 10.000 m
Engine: NumPy+rasterio
[1/36] div 00: new_div_label_0.tif -> LM_div36_div00.csv
[2/36] div 01: new_div_label_1.tif -> LM_div36_div01.csv
[3/36] div 02: new_div_label_2.tif -> LM_div36_div02.csv
[4/36] div 03: new_div_label_3.tif -> LM_div36_div03.csv
[5/36] div 04: new_div_label_4.tif -> LM_div36_div04.csv
[6/36] div 05: new_div_label_5.tif -> LM_div36_div05.csv
[7/36] div 06: new_div_label_6.tif -> LM_div36_div06.csv
[8/36] div 07: new_div_label_7.tif -> LM_div36_div07.csv
[9/36] div 08: new_div_label_8.tif -> LM_div36_div08.csv
[10/36] div 09: new_div_label_9.tif -> LM_div36_div09.csv
[11/36] div 10: new_div_label_10.tif -> LM_div36_div10.csv
[12/36] div 11: new_div_label_11.tif -> LM_div36_div11.csv
[13/36] div 12: new_div_label_12.tif -> LM_div36_di